# Decision Trees & Random Forests from Scratch

**Welcome, St. Mark!** Now we explore tree-based methods. Think of this as building diagnostic decision trees - like the flowchart doctors use to diagnose illnesses.

We'll explore:

1. **Decision Tree Construction** - Building trees through recursive splitting
2. **Splitting Criteria** - Gini impurity and information gain
3. **Random Forests** - Ensemble learning through bagging
4. **Feature Importance** - Understanding which features matter most

By the end, you'll understand how ensemble methods achieve remarkable accuracy.

## Core Definitions and Mathematical Foundations

### Fundamental Definitions

**Decision Tree:** A non-parametric supervised learning algorithm that partitions the feature space into regions and assigns each region a constant prediction value. Unlike parametric models (like logistic regression), decision trees adapt their complexity to the data structure without assuming any specific form of the relationship between features and target.

**Key Components of a Decision Tree:**
- **Root Node:** The topmost node representing the entire training dataset and the first decision point
- **Internal Nodes:** Nodes that split data based on a feature threshold, creating decision branches
- **Leaf Nodes:** Terminal nodes that provide final predictions (majority class for classification)
- **Branches:** Decision paths connecting nodes based on whether feature values are ≤ or > a threshold

**Random Forest:** An ensemble learning method that combines multiple decision trees through bootstrap aggregation (bagging) and feature randomness to reduce overfitting and improve generalization. The final prediction is typically the majority vote across all trees.

### Mathematical Foundations

**Gini Impurity:** A quadratic measure of node impurity that quantifies how "mixed" a node is with respect to class labels.

$$\text{Gini}(D) = 1 - \sum_{i=1}^{c} p_i^2$$

Where:
- $D$: Dataset at current node
- $c$: Number of classes
- $p_i$: Proportion of samples belonging to class $i$

**Intuition:** Lower Gini = more pure node (better for splitting). A Gini of 0 means all samples in the node belong to the same class (perfect purity), while a Gini of 0.5 (for binary classification) represents maximum impurity (50-50 class distribution).

**Entropy:** An alternative logarithmic measure of information content and impurity.

**Mathematical Formula:**
$$\text{Entropy}(D) = -\sum_{i=1}^{c} p_i \log_2(p_i)$$

**Information Gain:** The reduction in impurity achieved by splitting a node on a particular feature. The algorithm selects the split that maximizes information gain.

$$\text{Information Gain}(D, A) = \text{Impurity}(D) - \sum_{v \in \text{values}(A)} \frac{|D_v|}{|D|} \cdot \text{Impurity}(D_v)$$

Where:
- $D$: Parent dataset
- $A$: Feature to split on
- $D_v$: Subset of $D$ where feature $A$ has value $v$

**CART Algorithm:** Classification and Regression Trees - the specific algorithm we implement, characterized by:
1. **Binary Splits:** Each internal node splits in exactly two child nodes
2. **Greedy Algorithm:** Selects best split at each step using information gain
3. **Impurity Minimization:** Uses Gini impurity (or optionally entropy) for split quality
4. **Stopping Criteria:** Maximum depth, minimum samples per split, or pure nodes

## The Big Picture

Decision trees are flowchart-like structures for classification:
- **Root Node:** Starting decision point
- **Internal Nodes:** Feature-based splits
- **Leaf Nodes:** Final predictions
- **Branches:** Decision paths based on feature thresholds

**Key Question:** How do we automatically grow decision trees from data?

## Data Preparation: Classification Dataset

We'll use the same classification data as logistic regression for comparison.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from collections import Counter
import time

# Use same data as logistic regression for comparison
X, y = make_classification(n_samples=1000,
                          n_features=4,
                          n_classes=2,
                          n_informative=3,
                          n_redundant=1,
                          n_clusters_per_class=1,
                          random_state=42)

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set: X={X_train.shape}, y={y_train.shape}")
print(f"Test set: X={X_test.shape}, y={y_test.shape}")
print(f"Class distribution: {Counter(y_train)}")
print(f"Feature ranges: min={X_train.min():.2f}, max={X_train.max():.2f}")

**Cell Analysis:** We've prepared our dataset.

- **Same data:** Allows direct comparison with logistic regression
- **Feature ranges:** Decision trees handle different scales naturally
- **Class balance:** Important for tree construction

**Healthcare Analogy:** Like preparing patient data for diagnostic flowchart creation.

**Reflection Question:** Why might decision trees handle mixed data types better than logistic regression?

## Method 1: Gini Impurity - Measuring Node Purity

**Enhanced Definition:** Gini impurity quantifies the "impurity" or "disorder" of a node by measuring how evenly classes are distributed. It's a quadratic measure that reaches minimum (0) for pure nodes and maximum (0.5 for binary) for perfectly balanced splits.

**Mathematical Formula:**
$$\text{Gini}(D) = 1 - \sum_{i=1}^{c} p_i^2$$

Where:
- $D$: Dataset at current node
- $c$: Number of classes  
- $p_i$: Proportion of class $i$ in dataset

**Why Gini over Entropy?:**
- **Computational efficiency:** Gini uses simple squaring vs entropy's logarithm
- **Similar performance:** Both measure impurity effectively in practice
- **C4.5 vs CART:** C4.5 uses entropy, CART (our algorithm) uses Gini

Lower Gini = more pure node (better for splitting).

In [ ]:
def gini_impurity(y):
    """
    Calculate Gini impurity for a node.
    
    Formula: Gini = 1 - Σ p_i² where p_i is proportion of class i
    
    Parameters:
    y: Array of class labels for samples in this node
    
    Returns:
    Gini impurity score (0 = pure, 0.5 = maximum impurity for binary)
    """
    if len(y) == 0:
        return 0.0
    
    # Count class frequencies
    class_counts = Counter(y)
    total_samples = len(y)
    
    # Calculate Gini impurity: 1 - Σp²
    gini = 1.0
    for count in class_counts.values():
        prob = count / total_samples
        gini -= prob ** 2
    
    return gini

def entropy(y):
    """
    Calculate entropy (alternative to Gini).
    
    Formula: Entropy = -Σ p_i * log₂(p_i)
    """
    if len(y) == 0:
        return 0.0
    
    class_counts = Counter(y)
    total_samples = len(y)
    
    # Calculate entropy with log base 2
    entropy_val = 0.0
    for count in class_counts.values():
        if count > 0:
            prob = count / total_samples
            entropy_val -= prob * np.log2(prob)
    
    return entropy_val

# Test impurity measures on different node compositions
# Pure nodes have Gini = 0 (perfect certainty)
# Mixed nodes have higher Gini (closer to 0.5)
test_cases = [
    ([0, 0, 0, 0], "Pure class 0 - Perfect diagnosis"),
    ([1, 1, 1, 1], "Pure class 1 - Confident prediction"),
    ([0, 1, 0, 1], "50-50 mix - Maximum uncertainty"),
    ([0, 0, 1], "Mostly healthy - Slight uncertainty")
]

print("Impurity Measures Comparison - Healthcare Diagnosis Analogy:")
print("=" * 65)
print(f"{'Node Composition':<15} {'Gini':<8} {'Entropy':<8} Diagnosis Certainty")
print("-" * 65)

for labels, description in test_cases:
    gini_val = gini_impurity(labels)
    entropy_val = entropy(labels)
    print(f"{description:<15} {gini_val:<8.3f} {entropy_val:<8.3f} {str(labels)}")

# Visualize how Gini and entropy behave as class proportions change
# This shows why Gini provides quadratic penalty for impurity
proportions = np.linspace(0.01, 0.99, 100)
gini_curve = [1 - p**2 - (1-p)**2 for p in proportions]
entropy_curve = [-p*np.log2(p) - (1-p)*np.log2(1-p) for p in proportions]

plt.figure(figsize=(10, 6))
plt.plot(proportions, gini_curve, 'b-', linewidth=2, label='Gini Impurity (Quadratic)')
plt.plot(proportions, entropy_curve, 'r-', linewidth=2, label='Entropy (Logarithmic)')
plt.xlabel('Proportion of Class 1 in Node')
plt.ylabel('Impurity Value')
plt.title('Gini vs Entropy: Different Penalty Functions for Impurity')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

**Cell Analysis - Detailed Gini Explanation:**

**Impurity Measures Interpretation:**
- **Gini = 0 (Pure nodes):** All samples belong to same class - diagnostic certainty
- **Gini = 0.5 (50-50 split):** Maximum uncertainty - coin flip diagnosis  
- **Gini > 0.5:** Imbalanced but multi-class mixing

**Mathematical Properties:**
- **Quadratic penalty:** Gini (p) for binary classes: 1 - p² - (1-p)² = 2p(1-p)
- **Maximum at p=0.5:** Gini reaches 0.5 for perfectly balanced classes
- **Fast computation:** No logarithm required, suitable for large datasets

**Healthcare Translation:** Like certainty in medical diagnosis:
- Pure node (Gini=0): "Patient definitely has Malaria"
- High Gini (0.5): "Need more tests, completely uncertain"
- Mixed Gini: "Might be Typhoid, but some Malaria indicators present"

**Computational Trade-offs:**
- **Gini:** Faster (squares vs logs), insensitive to class probability extremes
- **Entropy:** More "aggressive" penalty for changes, theoretically sounder in information theory

**Reflection Question:** Why might the quadratic penalty in Gini be more practical for medical decision trees than logarithmic entropy?

## Method 2: Finding Optimal Splits - Information Gain

**Enhanced Definition:** We evaluate every possible feature split to find the one that maximizes information gain - the reduction in impurity achieved by partitioning the data.

**Information Gain Formula:**
$$\text{Information Gain}(D, A) = \text{Impurity}(D) - \sum_{v \in \text{values}(A)} \frac{|D_v|}{|D|} \cdot \text{Impurity}(D_v)$$

Where:
- $D$: Parent dataset (current node)
- $A$: Feature to split on  
- $D_v$: Data subset where feature $A$ has value $v$
- The weighted average impurity of children is subtracted from parent impurity

**Greedy Algorithm Concept:** Higher information gain = better split. The CART algorithm tries every possible feature and threshold combination to maximize this gain.

**Why Midpoints Between Values?** The algorithm evaluates splits at the midpoint between consecutive sorted feature values. This creates binary splits that are:
- **Deterministic:** Every sample goes left or right based on threshold
- **Efficient:** Only considers points where splits can actually change
- **Robust:** Avoids splits within identical feature values

In [ ]:
def find_best_split(X, y, feature_indices=None):
    """
    Find the best feature and threshold to split on using information gain maximization.
    
    This implements the CART algorithm's greedy split selection strategy.
    
    Parameters:
    X: Feature matrix
    y: Target labels 
    feature_indices: Subset of features to consider (for random forests)
    
    Returns:
    best_feature, best_threshold, best_gain, best_splits
    """
    if feature_indices is None:
        feature_indices = range(X.shape[1])
    
    best_gain = 0  # Track the maximum information gain found
    best_feature = None  # Feature index for best split
    best_threshold = None  # Threshold value for best split
    best_splits = None  # (left_labels, right_labels) for best split
    
    # Calculate parent node impurity once
    parent_impurity = gini_impurity(y)
    n_samples = len(y)
    
    # Greedy algorithm: Try all possible splits systematically
    for feature_idx in feature_indices:
        feature_values = X[:, feature_idx]
        unique_values = np.unique(feature_values)
        
        # CART strategy: Create thresholds at midpoints between consecutive values
        # This ensures every sample goes deterministically left or right
        sorted_values = np.sort(unique_values)
        thresholds = (sorted_values[:-1] + sorted_values[1:]) / 2
        
        for threshold in thresholds:
            # Binary split: samples ≤ threshold go left, > threshold go right
            left_mask = feature_values <= threshold
            right_mask = ~left_mask
            
            y_left = y[left_mask]
            y_right = y[right_mask]
            
            # Skip trivial splits (empty child nodes)
            if len(y_left) == 0 or len(y_right) == 0:
                continue
            
            # Calculate weighted average impurity of child nodes
            left_impurity = gini_impurity(y_left)
            right_impurity = gini_impurity(y_right)
            
            left_weight = len(y_left) / n_samples
            right_weight = len(y_right) / n_samples
            
            # Weighted average = (weight_left × impurity_left) + (weight_right × impurity_right)  
            weighted_child_impurity = (left_weight * left_impurity + 
                                      right_weight * right_impurity)
            
            # Information gain = parent_impurity - weighted_child_impurity
            # Higher gain = better split (more impurity reduction)
            information_gain = parent_impurity - weighted_child_impurity
            
            # Update best split if this split achieves highest gain
            if information_gain > best_gain:
                best_gain = information_gain
                best_feature = feature_idx
                best_threshold = threshold
                best_splits = (y_left, y_right)
    
    return best_feature, best_threshold, best_gain, best_splits

# Demonstrate how split finding works with information gain
feature_idx, threshold, gain, splits = find_best_split(X_train, y_train)

print(f"🔍 Optimal Split Found Using Information Gain:")
print(f"Selected Feature: {feature_idx}")
print(f"Split Threshold: {threshold:.3f} (values ≤ {threshold:.3f} go left)")
print(f"Information Gain Achieved: {gain:.4f}")
print(f"Left Child: {len(splits[0])} samples, Gini: {gini_impurity(splits[0]):.3f}, Classes: {Counter(splits[0])}")
print(f"Right Child: {len(splits[1])} samples, Gini: {gini_impurity(splits[1]):.3f}, Classes: {Counter(splits[1])}")
print(f"Total Child Gini (weighted): {(len(splits[0])/len(y_train)*gini_impurity(splits[0]) + len(splits[1])/len(y_train)*gini_impurity(splits[1])):.3f}")

# Visualize the information gain maximization in action
plt.figure(figsize=(12, 8))

# Plot all data points with class colors
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, alpha=0.6, cmap='RdYlBu', s=50)

# Highlight the chosen split feature and threshold
# This split maximizes information gain (minimizes weighted child impurity)
if feature_idx == 0:
    plt.axvline(x=threshold, color='red', linewidth=3, label=f'Information Gain Split: Feature {feature_idx} ≤ {threshold:.3f}')
    # Add annotation for information gain
    plt.annotate(f'Information Gain: {gain:.3f}\n(split maximizes purity improvement)', 
                xy=(threshold, 0.5), xycoords='data',
                xytext=(10, 10), textcoords='offset points',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.8),
                fontsize=9, ha='left')
else:
    plt.axhline(y=threshold, color='red', linewidth=3, label=f'Information Gain Split: Feature {feature_idx} ≤ {threshold:.3f}')
    plt.annotate(f'Information Gain: {gain:.3f}\n(split maximizes purity improvement)', 
                xy=(threshold, 0.5), xycoords='data',
                xytext=(10, 10), textcoords='offset points',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.8),
                fontsize=9, ha='left')

plt.xlabel('Feature 0')
plt.ylabel('Feature 1') 
plt.title('Decision Tree: Information Gain Maximization\\n(Red line shows split that maximizes impurity reduction)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

**Cell Analysis - Information Gain Deep Dive:**

**Algorithm Mechanics:**
- **Exhaustive Search:** Systematically evaluates all features and possible thresholds
- **Midpoint Strategy:** Splits at midpoints between consecutive feature values
- **Gain Calculation:** Measures how much weighted impurity decreases after split
- **Greedy Selection:** Chooses split that maximizes information gain

**Mathematical Rationale for Midpoint Thresholds:**
- **Avoids no-change splits:** Splitting within a cluster of identical values has no effect
- **Binary decisions:** Each sample definitively goes left or right
- **Efficiency:** Reduces number of candidate splits to O(n log n) vs O(n²)
- **Determinism:** Eliminates ambiguity in split boundaries

**Why Midpoints Matter in Healthcare:**
- **Quantitative features:** Lab values, vital signs have natural split points
- **Binary decisions:** "Is blood pressure ≤ 140?" vs "Is it ≤ 138 or ≤ 142?"
- **Clinical thresholds:** Often based on standard cutoffs in medical guidelines

**Computational Complexity:**
- **Per-node cost:** O(F × V × N) where F=features, V=unique values, N=samples
- **CART optimization:** Midpoint strategy reduces V to O(N) in practice
- **Trade-off:** Computationally expensive but finds optimal splits

**Healthcare Interpretation:** Like choosing the most diagnostic question:
- "Does patient have fever > 38°C?" vs "Does patient have fever > 37°C?"
- Information gain quantifies how well the question separates diseases

**Reflection Question:** Why would midpoints between feature values create more deterministic decision boundaries than arbitrary thresholds?